### 引用R包

In [1]:
from rpy2.robjects.packages import importr
from rpy2.robjects import FloatVector, StrVector, r
import pandas as pd
ci = importr("CausalImpact")
zoo = importr("zoo")
base = importr("base")

### 加载game_churn的数据，并进行初步转换

In [2]:
gc_data = pd.read_csv("http://image.cador.cn/data/game_churn.csv")
gc_data.y = [x.split('%')[0] for x in gc_data.y.values]
gc_data.orent = [x.split('%')[0] for x in gc_data.orent.values]
time_points = base.seq_Date(base.as_Date('2019-04-01'), by=1, length_out=25)
data = zoo.zoo(base.cbind(FloatVector(gc_data.y), FloatVector(gc_data.cpi), FloatVector(gc_data.orent)), time_points)
pre_peroid = base.as_Date(StrVector(['2019-04-01', '2019-04-15']))
post_peroid = base.as_Date(StrVector(['2019-04-16', '2019-04-25']))

### 建立时序因果推断模型，并得出分析图表

In [3]:
model_args = r("list(niter = 10000, nseasons = 7, season.duration = 1)")
impact = ci.CausalImpact(data, pre_peroid, post_peroid, model_args=model_args)
r('png("causalimpact.jpg")')
print(ci.plot_CausalImpact(impact))
r('dev.off()')

1


![](causalimpact.jpg)

### 查看因果推断的报告

In [4]:
r('sink("report.txt")')
ci.PrintReport(impact)
r('sink()')

rpy2.rinterface.NULL

In [5]:
with open("report.txt", 'r') as f:
    print(f.read())

Analysis report {CausalImpact}


During the post-intervention period, the response variable had an average value of approx. 43.18. By contrast, in the absence of an intervention, we would have expected an average response of 47.37. The 95% interval of this counterfactual prediction is [45.28, 49.47]. Subtracting this prediction from the observed response yields an estimate of the causal effect the intervention had on the response variable. This effect is -4.18 with a 95% interval of [-6.29, -2.09]. For a discussion of the significance of this effect, see below.

Summing up the individual data points during the post-intervention period (which can only sometimes be meaningfully interpreted), the response variable had an overall value of 431.84. By contrast, had the intervention not taken place, we would have expected a sum of 473.68. The 95% interval of this prediction is [452.78, 494.73].

The above results are given in terms of absolute numbers. In relative terms, the response variable